In [1]:
# The code was removed by Watson Studio for sharing.

In [2]:
#Importar las bibliotecas que serán necesarias
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [3]:
# The code was removed by Watson Studio for sharing.

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,target
0,-0.458258,-0.579012,2.886009,-0.075516,1.674060,-2.431912,0.534850,-0.846473,2
1,1.626615,-0.028332,-1.443184,-1.007447,-0.989093,1.202627,-0.038211,-1.298616,0
2,1.721356,-0.137676,-1.322593,-1.333187,-0.723713,0.843285,-0.588495,0.066682,0
3,-0.715103,3.265915,-0.791030,-2.967881,2.497628,-0.277387,-0.551611,0.668354,2
4,2.944098,0.197871,-2.257025,1.032789,-4.104693,4.716115,-0.380813,-1.393825,0


In [4]:
#sp: Crear el marco de datos de prueba
#EN: Create test dataframe
body = client_1731e53444e54241a5f4c7fd9b23b87b.get_object(Bucket='nuwechallenges-donotdelete-pr-vqtclvgfnzr8if',Key='uoc_X_test.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

x_test = pd.read_csv(body)
x_test.head()


,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8
0,0.052199,2.514513,-0.197672,1.978709,-2.014691,3.852886,-2.126254,0.794624
1,-0.828073,0.085895,1.712522,0.078828,1.280057,-1.462486,2.469702,0.816540
2,-1.086411,-0.142109,0.013089,-0.115223,0.948242,-1.121574,0.517415,0.250672
3,0.093129,2.868934,-0.075120,0.650641,-0.886186,2.917352,-0.722935,0.063071
4,0.046167,2.822612,0.433869,-3.054525,2.537684,-0.717312,0.862266,-0.525491


In [5]:
#Veamos el tamaño total de nuestro marco de datos
df_train.shape

(2100, 9)

In [6]:
#Compruebo que no faltan datos
df_train.isnull().sum()

feature1    0
feature2    0
feature3    0
feature4    0
feature5    0
feature6    0
feature7    0
feature8    0
target      0
dtype: int64

In [7]:
#Compruebo el tipo de datos
df_train.dtypes

feature1    float64
feature2    float64
feature3    float64
feature4    float64
feature5    float64
feature6    float64
feature7    float64
feature8    float64
target        int64
dtype: object

In [8]:
#Vemos la cantidad de resultados en target previo a definirlos
df_train['target'].value_counts()

0    713
1    705
2    682
Name: target, dtype: int64

## Descripción de las etiquetas

<li> Buena calidad del aire (target 0): 713 observaciones. </li>
<li> Calidad moderada del aire (target 1): 705 observaciones. </li>
<li> Mala calidad del aire (target 3): 682 observaciones. </li>

In [9]:
#Ahora procedo a dividir el set de datos de prueba en variables independientes(x) y variable predictora(y)
X_train = df_train[['feature1', 'feature2', 'feature3', 'feature4', 'feature5', 'feature6', 'feature7', 'feature8']]
X_train.shape

(2100, 8)

In [10]:
Y_train = df_train[['target']]
Y_train.head()

,target
0,2
1,0
2,0
3,2
4,0


In [11]:
#Con el set de prueba listo, procedo a estandarizar 
scaler = StandardScaler()
X = scaler.fit_transform(X_train)

#Numpy array para y
y = Y_train.to_numpy()

In [12]:
#Es momento de buscar los mejores parámetros para RandomForest
rand_for = RandomForestClassifier(n_jobs=-1,max_features= 'sqrt' ,n_estimators=50, oob_score = True) 

param_grid = { 
    'n_estimators': [200, 700],
    'max_features': ['auto', 'sqrt', 'log2']
}

CV_rfc = GridSearchCV(estimator=rand_for, param_grid=param_grid, cv= 5)
CV_rfc.fit(X, np.ravel(y))
best_params = CV_rfc.best_params_

In [13]:
print(best_params)

{'max_features': 'log2', 'n_estimators': 700}


In [14]:
#Escalar el set de prueba
x_ = scaler.fit_transform(x_test)

#Ajustar los parámetros para hacer la predicción
rfc = RandomForestClassifier(n_jobs=-1,max_features= 'log2' ,n_estimators=700, oob_score = True)
rfc.fit(X, np.ravel(y))
yhat = rfc.predict(x_)

print(yhat)

[0 2 1 0 2 1 2 0 1 1 0 1 1 1 2 2 2 1 1 0 0 0 1 0 0 1 1 1 1 2 0 1 2 1 2 0 0
 2 1 0 1 1 2 1 1 0 0 0 1 2 2 0 1 0 0 2 0 2 2 0 1 0 2 0 1 0 1 2 1 0 0 0 2 0
 2 0 0 0 1 1 2 2 2 1 2 0 0 2 0 1 0 0 0 0 0 2 2 1 1 2 0 0 0 2 0 2 2 1 0 1 2
 0 1 2 2 1 2 2 1 0 0 1 2 1 1 1 2 1 1 1 1 1 2 0 0 1 2 0 0 2 0 2 2 1 0 2 1 2
 1 2 1 1 1 2 0 2 1 1 2 1 0 1 2 1 0 2 0 2 1 0 1 1 0 2 0 2 2 0 0 2 1 1 0 1 0
 1 0 2 1 1 0 0 1 2 2 0 2 2 1 1 0 2 2 2 1 1 2 1 0 2 0 0 1 1 1 0 1 1 0 1 1 1
 1 0 2 2 1 1 1 2 0 1 2 1 2 0 1 2 2 1 0 2 2 0 1 0 0 0 1 2 1 2 1 2 1 0 0 1 1
 2 1 2 0 2 1 2 0 2 0 1 1 2 1 2 1 0 0 2 1 2 1 1 1 0 1 2 1 0 1 1 0 1 0 1 0 0
 1 2 0 2 1 2 0 0 2 2 2 0 1 1 0 2 1 1 0 0 2 1 2 2 0 1 0 2 2 0 2 1 2 1 2 1 1
 2 0 1 1 0 1 1 1 1 2 0 0 1 2 2 1 0 0 2 2 1 1 1 2 2 2 1 2 2 0 1 0 2 0 1 1 2
 2 2 0 1 2 2 0 1 2 0 2 0 1 0 1 1 2 1 1 2 2 1 1 2 0 2 1 2 0 2 2 2 2 2 0 0 2
 0 0 2 0 1 1 1 2 2 1 0 2 2 0 0 1 0 1 1 1 1 1 1 0 1 2 0 1 1 2 0 2 0 1 1 1 2
 0 2 1 2 2 2 1 0 0 0 2 0 1 2 0 2 1 2 0 2 0 2 1 0 1 0 2 0 2 2 0 1 1 2 0 2 0
 1 2 0 2 0 1 2 2 0 2 2 0 

In [17]:
# The code was removed by Watson Studio for sharing.

{'file_name': 'predicciones.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'nuwechallenges-donotdelete-pr-vqtclvgfnzr8if',
 'asset_id': '1f8203df-9ea4-4737-92ba-7c632e5ec0c5'}

In [ ]:
#Guardar las predicciones en csv.
project.save_data(data=pd.DataFrame(yhat, columns=['Predictions']).to_csv(index=False),file_name='predicciones.csv',overwrite=True)